In [1]:
# Author: Joshua Gregory
# Date: Sept. 2025
# Version 1.0: Uses BFS & Iterative DFS to attempt to find path to goal state in 8-Puzzle Problem variant.
# Version 2.0: Implements A* with cost function f(n) = g(n) + h(n) where:
    # f(n) = estimated cost of cheapest solution through n
    # g(n) = cost to reach goal node from start state
    # h(n) = Manhattan Distance of each tile from its proper position

In [73]:
import random                  # for 8-puzzle grid generation
from collections import deque  # for BFS
import heapq                   # for A*
from itertools import count    # for A*

In [3]:
# Global fields
GOAL_STATE = [[1,2,3], [8,0,4], [7,6,5]]  # solution state as 2D array
GRID_SIZE = 3
row_moves = [0, 0, -1, 1]  # L/R/U/D directional values for rows  *Adapted from similar Java ops on Stack Overflow
col_moves = [-1, 1, 0, 0]  # L/R/U/D directional values for cols  *Adapted from similar Java ops on Stack Overflow

In [61]:
# Layer = 8-Puzzle state space construction (as 2d arr: [[0,1,2], [3,4,5], [6,7,8]])
class Eight_Puzzle:
  def __init__(self, grid_state, blank_x, blank_y, depth_level):
    """
    Eight_Puzzle constructor to define & initialize object fields.
    @param grid_state: attribute for 2d array (grid) puzzle state.
    @param blank_x: attribute for row position of blank tile.
    @param blank_y: attribute for column position of blank tile.
    @param depth_level: attribute for depth of current state node.
    """
    self.grid_state = grid_state    # puzzle state 2d array grid
    self.blank_x = blank_x          # row coord of blank tile (value 0)
    self.blank_y = blank_y          # col coord of blank tile (value 0)
    self.depth_level = depth_level  # depth relative to root

  def to_grid(self):
    """
    Prints current grid state as a grid with underscore for blank tile.
    """
    print("--------")
    for row in self.grid_state:
      print(' '.join('_' if val == 0 else str(val) for val in row))
    print("--------")

  def is_goal(self):
    """
    Checks if input 2d array (grid state) matches GOAL_STATE.
    @return: (Boolean) T if all elements in input 2d array match GOAL_STATE, else F.
    """
    return self.grid_state == GOAL_STATE

  @staticmethod
  def adj_in_range(adj_x, adj_y):
    """
    Validates that a tile position is in grid state range.
    @param adj_x: Row position of target tile.
    @param adj_y: Col position of target tile.
    @return: (Boolean) T if tile is in grid state range, else F.
    """
    return 0 <= adj_x < GRID_SIZE and 0 <= adj_y < GRID_SIZE

  def move_blank_tile(self, adj_x, adj_y):
    """
    Updates grid state by swapping blank tile with validated neighbor tile.
    @param adj_x: Row position of target tile.
    @param adj_y: Col position of target tile.
    @return: Updated state with swapped tiles.
    """
    new_state = [row[:] for row in self.grid_state]  # copy input grid state

    temp = new_state[self.blank_x][self.blank_y]                     # store in temp var
    new_state[self.blank_x][self.blank_y] = new_state[adj_x][adj_y]  # swap with adj tile
    new_state[adj_x][adj_y] = temp                                   # swap with blank tile

    return Eight_Puzzle(new_state, adj_x, adj_y, self.depth_level+1) # return updated state

In [54]:
# Layer = Data (store 8-puzzle object) --> Eight_Puzzle
def get_root_grid():
  """
  Creates grid for 8-puzzle root state as a 2d array with elements [0,8] shuffled.
  @return: Root grid for 8-puzzle, row & column coords of blank tile.
  """
  tile_values = list(range(9))  # list of integers [0,8], 0 = blank tile
  random.shuffle(tile_values)   # shuffle tile positions

  root_grid = [tile_values[i : i+3] for i in range(0, 9, GRID_SIZE)]  # convert to 3x3 grid

  for blank_x in range(GRID_SIZE):
    for blank_y in range(GRID_SIZE):          # search each col in each row
      if root_grid[blank_x][blank_y] == 0:    # for blank tile with value 0
        return root_grid, blank_x, blank_y    # return grid & xy coords of blank tile

In [68]:
# Layer = BFS Traversal --> Eight_Puzzle
def run_bfs(grid_state, blank_x, blank_y):
  """
  Searches tiers completely before visiting children nodes.
  @param grid_state: Puzzle state as 2d array via get_root_grid().
  @param blank_x: Row position of blank tile (value 0).
  @param blank_y: Column position of blank tile (value 0).
  """
  Q = deque()   # FIFO queue for searching complete tiers
  SEEN = set()  # re-init per instance, prevents cycling

  root = Eight_Puzzle(grid_state, blank_x, blank_y, 0)  # create root obj
  Q.append(root) # enqueue root
  SEEN.add(tuple(map(tuple, grid_state)))  # 2d array state = immutable tuple of tupled values ((r1),(r2),(r3))

  while Q:                  # while states remain in queue
    current = Q.popleft()   # pop & store first (leftmost) node

    print(f"\nDepth = {current.depth_level}:")  # print current depth
    current.to_grid()                     # print NxN grid

    if current.is_goal():
      print(f"\n~ Goal state reached at depth {current.depth_level} via BFS! ~")  # solution state reached
      return  # exit

    for i in range(4):  # loop row_moves & col_moves while i = [0,3]
      adj_x = current.blank_x + row_moves[i]  # get row coords for possible blank tile position
      adj_y = current.blank_y + col_moves[i]  # get col coords

      if Eight_Puzzle.adj_in_range(adj_x, adj_y):  # validate xy in range
        new_state = current.move_blank_tile(adj_x, adj_y)  # move blank tile
        state_tuple = tuple(map(tuple, new_state.grid_state))  # 2d array = hashable tuple of tuples

        if state_tuple not in SEEN:  # check if new_state has been visited
          SEEN.add(state_tuple)      # add unvisited state to set
          Q.append(new_state)        # enqueue obj with new blank tile & iterated to next depth

  print("~ Goal state unreachable from root state via BFS. ~")  # all possible moves exhausted

In [65]:
# Layer = Iterative DFS Traversal --> Eight_Puzzle
def run_idfs(grid_state, blank_x, blank_y):
  """
  Searches branches completely before backtracking to sibling.
  @param grid_state: Puzzle state as 2d array via get_root_grid().
  @param blank_x: Row position of blank tile (value 0).
  @param blank_y: Column position of blank tile (value 0).
  """
  S = []        # LIFO stack to explore branch depth iteratively
  SEEN = set()  # re-init per instance, stores seen states to prevent cycling

  root = Eight_Puzzle(grid_state, blank_x, blank_y, 0)  # create obj
  S.append(root)  # push obj to stack
  SEEN.add(tuple(map(tuple, grid_state)))  # convert to hashable tuple, add to seen nodes

  while S:             # while states remain on stack
    current = S.pop()  # pop most recently added state

    print(f"\nDepth = {current.depth_level}:")  # print current depth
    current.to_grid()                     # print state as NxN grid

    if current.is_goal():   # compare to goal state
      print(f"\n~ Goal state reached at depth {current.depth_level} via iDFS! ~")  # goal state reached
      return  # exit function

    for i in range(4):  # loop through row_moves & col_moves
      adj_x = current.blank_x + row_moves[i]  # new x = current + L/R/U/D
      adj_y = current.blank_y + col_moves[i]  # new y = current + L/R/U/D

      if Eight_Puzzle.adj_in_range(adj_x, adj_y):  # validate xy in range
        new_state = current.move_blank_tile(adj_x, adj_y)  # move blank tile
        state_tuple = tuple(map(tuple, new_state.grid_state))  # 2d array = hashable tuple of tuples

        if state_tuple not in SEEN:  # check if new_state has been visited
          SEEN.add(state_tuple)      # add unvisited state to set
          S.append(new_state)        # add obj to stack with new blank tile & iterated to next depth

  print("\n~ Goal state unreachable from root state via iDFS. ~")  # all possible moves exhausted

In [76]:
# Layer = A* Traversal --> Eight_Puzzle
def run_astar(grid_state, blank_x, blank_y):
  """
  Cost function: f(n) = g(n) + h(n), where: f(n) = g(n) + h(n), estimated cost of cheapest solution through n,
  g(n) = the cost to reach the goal node from starting state, and
  h(n) = Manhattan Distance of each tile from its proper position.
  Ie: the estimated cost of cheapest solution = cost to goal + Manhattan distance per tile.
  """
  PQUE = []          # priority queue min-heap
  SEEN = set()       # re-init per instance, set of visited states
  counter = count()  # tie-breaker for heap entries

  root = Eight_Puzzle(grid_state, blank_x, blank_y, 0)  # set root to new puzzle obj
  heapq.heappush(PQUE, (get_manhattan_distance(grid_state), next(counter), root))
  SEEN.add(tuple(map(tuple, grid_state)))

  while PQUE:
    _, _, current = heapq.heappop(PQUE)

    print(f"\nDepth = {current.depth_level}:")
    current.to_grid()

    if current.is_goal():
      print(f"\n~ Goal state reached at depth {current.depth_level}! ~")
      return

    for i in range(4):
      adj_x = current.blank_x + row_moves[i]
      adj_y = current.blank_y + col_moves[i]

      if Eight_Puzzle.adj_in_range(adj_x, adj_y):  # validate xy in range
        new_state = current.move_blank_tile(adj_x, adj_y)  # move blank tile
        state_tuple = tuple(map(tuple, new_state.grid_state))  # 2d array = hashable tuple of tuples

        if state_tuple not in SEEN:
          SEEN.add(state_tuple)       # add to set
          g = new_state.depth_level   # update g
          h = get_manhattan_distance(new_state.grid_state)  # update h
          f = g + h                   # update cost function
          heapq.heappush(PQUE, (f, next(counter), new_state))  # push to min-heap

  print("\n~ Goal state unreachable from root state via A*. ~")

def get_manhattan_distance(grid_state):
  """
  Calculates Manhattan heuristic per grid instance.
  @param grid_state: 2d arrray representation of current puzzle state.
  @return: (int) sum of distances of each tile from goal positions.
  """
  m_dist = 0  # init distance to 0
  for i in range(GRID_SIZE):  # iterate through grid rows
    for j in range(GRID_SIZE):  # iterate through grid cols
      value = grid_state[i][j]  # value of each tile
      if value != 0:
        for goal_i in range(GRID_SIZE):  # for goal row pos
          for goal_j in range(GRID_SIZE):  # for goal col pos
            if GOAL_STATE[goal_i][goal_j] == value:
              m_dist += abs(i - goal_i) + abs(j - goal_j)  # calculate distance
              break
  return m_dist  # return calculated distance

In [43]:
# Layer = UI Driver
def main():
  root_puzzle = None  # initialize puzzle object to null to check for

  while True:
    choice = get_menu_choice()  # menu prompt for validated input

    # MENU 1: GENERATE NEW 8-PUZZLE GRID
    if choice == '1':
      print("Generating new 8-puzzle grid...")
      grid, blank_x, blank_y = get_root_grid()  # get grid & xy coords of blank tile (value=0)
      root_puzzle = Eight_Puzzle(grid, blank_x, blank_y, 0)  # construct 8-Puzzle object

      print("\nNew 8-Puzzle root state:")
      root_puzzle.to_grid()  # print obj as NxN grid

    # MENU 2: SOLVE VIA BFS
    if choice == '2':
      if not root_puzzle:  # if puzzle not created first
        print("ERROR: Requires an 8-Puzzle grid (use menu option 1).")
      else:
        print()  # formatting
        run_bfs(root_puzzle.grid_state, root_puzzle.blank_x, root_puzzle.blank_y)  # input = return from get_root_grid()

    # MENU 3: SOLVE VIA iDFS
    if choice == '3':
      if not root_puzzle:  # if new puzzle not created
        print("ERROR: Requires an 8-Puzzle grid (use menu option 1).")
      else:
        print()  # formatting
        run_idfs(root_puzzle.grid_state, root_puzzle.blank_x, root_puzzle.blank_y)  # input = return from get_root_grid()

    if choice == '4':
      if not root_puzzle:  # if new puzzle not created
        print("ERROR: Requires an 8-Puzzle grid (use menu option 1).")
      else:
        print()  # formatting
        run_astar(root_puzzle.grid_state, root_puzzle.blank_x, root_puzzle.blank_y)  # input = return from get_root_grid()

    # MENU 5: EXIT
    if choice == '5':
      print("Exiting program... Goodbye!")
      return

# Helper method to display menu & return validated user input
def get_menu_choice():
  while True:  # loop menu prompt until valid input
    print("\n+------------------------------+")
    print("| 8-PUZZLE BFS/iDFS/A* SOLVER  |")
    print("+------------------------------+")
    print("| 1. New 8-Puzzle Grid         |")
    print("| 2. Run BFS Path Finder       |")
    print("| 3. Run IDFS Path Finder      |")
    print("| 4. Run A* Path Finder        |")
    print("| 5. Exit Program              |")
    print("+------------------------------+")

    choice = input("\nEnter a menu option: ").strip()  # save input without spaces

    if choice == '1' or choice == '2' or choice == '3' or choice == '4' or choice == '5':
      return choice
    else:
      print("ERROR: Invalid input (enter menu option 1, 2, 3, or 4).")

In [ ]:
# Layer = Entry Point
main()

Streaming output truncated to the last 5000 lines.
1 3 6
8 7 2
_ 4 5
--------

Depth = 12:
--------
1 3 8
6 7 _
4 2 5
--------

Depth = 12:
--------
1 3 8
6 2 7
4 _ 5
--------

Depth = 14:
--------
2 6 3
1 8 _
4 5 7
--------

Depth = 12:
--------
6 7 2
1 3 _
8 4 5
--------

Depth = 12:
--------
6 _ 2
1 7 5
8 3 4
--------

Depth = 15:
--------
2 8 4
1 6 3
5 7 _
--------

Depth = 15:
--------
2 8 4
1 _ 3
5 6 7
--------

Depth = 15:
--------
6 2 4
8 _ 3
1 5 7
--------

Depth = 15:
--------
6 2 4
1 8 3
_ 5 7
--------

Depth = 16:
--------
1 3 8
6 4 2
7 _ 5
--------

Depth = 16:
--------
3 8 2
1 4 6
7 _ 5
--------

Depth = 16:
--------
3 8 2
1 6 5
7 _ 4
--------

Depth = 13:
--------
1 6 8
4 2 3
5 7 _
--------

Depth = 13:
--------
1 6 2
4 3 8
5 7 _
--------

Depth = 13:
--------
1 6 2
4 _ 8
3 7 5
--------

Depth = 14:
--------
8 7 3
_ 2 6
1 4 5
--------

Depth = 14:
--------
8 _ 3
2 7 6
1 4 5
--------

Depth = 14:
--------
8 7 3
2 4 6
1 _ 5
--------

Depth = 14:
--------
8 7 3
2 6 5
1 _ 4
